# Install library

In [ ]:
!pip install -q datasets==2.16.0
!pip install -q bitsandbytes
!pip install -q tiktoken
!pip install -q peft
!pip install -q trl
!pip install -q transformers
!pip install -q openpyxl
!pip install -q pandas
!pip install -q scikit-learn
!pip install -q flash-attn
#pip install -q transformers==4.38.1

# Import library

In [ ]:
import json
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from accelerate import PartialState
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from peft import prepare_model_for_kbit_training
from transformers import TrainingArguments

# Hyperparameters

In [ ]:
modelpath = "Qwen/Qwen2-1.5B-Instruct"
lr=2e-4      # learning rate
bs=16            # batch size
bs_eval=16      # batch size for evals
ga_steps=1     # gradient acc. steps
epochs=4
max_length=128      # max. sample length with 24GB VRAM
output_dir="out"

# Remove old model
Because of limited storage, we can't save all models, we need to delete all models in cache by the following code:
- rm -r out: delete out folder (because I save finetuned model in out folder), you can change folder name like (rm -r output_folder). If you doesn't have out folder, this commend do not thing.
- all pretrained huggingface models will auto save in transformers.TRANSFORMERS_CACHE. I use shutil.rmtree to delete them.

In [ ]:
!rm -r out

In [ ]:
# from transformers import TRANSFORMERS_CACHE
# print(TRANSFORMERS_CACHE)

# import shutil
# shutil.rmtree(TRANSFORMERS_CACHE)

# Create Dataset
Download dataset from [kaggle synthetic-vietnamese-students-feedback-corpus](https://www.kaggle.com/datasets/toreleon/synthetic-vietnamese-students-feedback-corpus/data)

We need convert DataFrame to json line (jsonl).

In [ ]:
df_train = pd.read_csv("synthetic_train.csv")
df_test = pd.read_csv("synthetic_val.csv")

In [ ]:
df_train.head()

,sentence,sentiment,topic
0,Đội ngũ bảo trì quá thưa thớt dẫn đến không đả...,negative,facility
1,The university's musical and artistic faciliti...,neutral,facility
2,Phương pháp giảng dạy phù hợp với các đối tượn...,neutral,curriculum
3,Chương trình học giúp tôi trở thành một chuyên...,positive,curriculum
4,Tôi nghĩ rằng chương trình đào tạo có thể có t...,neutral,curriculum


In [ ]:
df_test.head()

,sentence,sentiment,topic
0,Chất lượng vật chất kém.,negative,facility
1,"Phần mềm học tập quá khó sử dụng, khiến sinh v...",negative,facility
2,Trường tôi thiếu những tiện ích cơ bản như máy...,negative,facility
3,Cần tạo thêm các hoạt động gắn kết giữa sinh v...,neutral,curriculum
4,Họ rất khoan dung và lượng giác trong quan điể...,neutral,others


In [ ]:
df_train.sentiment.value_counts()

sentiment
neutral     2724
negative    2711
positive    2709
Name: count, dtype: int64

In [ ]:
df_test.sentiment.value_counts()

sentiment
negative    686
positive    680
neutral     670
Name: count, dtype: int64

In [ ]:
df_train['len'] = df_train.sentence.apply(lambda x: len(str(x).split()))
df_test['len'] = df_test.sentence.apply(lambda x: len(str(x).split()))

In [ ]:
df_train['len'].describe()

count    8144.000000
mean       15.549730
std         5.018764
min         3.000000
25%        12.000000
50%        15.000000
75%        18.000000
max        43.000000
Name: len, dtype: float64

In [ ]:
df_test['len'].describe()

count    2036.000000
mean       15.694990
std         5.185957
min         2.000000
25%        12.000000
50%        15.000000
75%        19.000000
max        48.000000
Name: len, dtype: float64

In [ ]:
with open('train.jsonl', 'w') as outfile:
    for i, x in df_train.iterrows():
        comment = x['sentence']
        label = x['sentiment']
        #label = 'yes' if label == 'relevance' else 'no'
        data = {
            "input": f'''The sentiment of this comment "{comment}" is''',
            "output": f"{label}"
        }
        json.dump(data, outfile)
        outfile.write('\n')

In [ ]:
with open('test.jsonl', 'w') as outfile:
    for i, x in df_test.iterrows():
        comment = x['sentence']
        label = x['sentiment']
        #label = 'yes' if label == 'relevance' else 'no'
        data = {
            "input": f'''The sentiment of this comment "{comment}" is''',
            "output": f"{label}"
        }
        json.dump(data, outfile)
        outfile.write('\n')

In [ ]:
data_files = {
    "train": "train.jsonl",
    "validation": "test.jsonl",
}

dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 8144
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 2036
    })
})

# Create prompt format
Load tokenizer by AutoTokenizer.from_pretrained:
- We need create and copy YOUR TOKEN from [huggingface](https://huggingface.co/settings/tokens)
- We need use padding_side = 'right' because training library need padding_side = 'right' when training. You can use left padding but you need to make sure the library is not corrupted and check whether performance is affected by left padding!
- If you have 1 prompt like "test thử mô hình" and want to tokenize it, just you tokenizer(prompt, return_tensors="pt"). You can see output of tokenizer in cell below (output includes input_ids (list index of each token in prompt) and attention mask)
- We can use tokenizer.batch_decode to see how tokenizer restore string from token tensor. You can see that it automatically adds the start token "<bos>" at the beginning of the string.
- To train llm, we only need to pass 1 sentence to llm (including input and desired output) without specifying which is the input and which is the output.
- I wrote the function formatting_prompts_func to convert input and output to prompt and tested this function, you can see below.
- When predicting, remove the output part to let the model predict itself. See the predict section below later.
- we only need to predict some next tokens like A. positive, and B. neutral. We don't care what the model says after sentiment. Then we do not need to add <eos token>. If you fine-tune the model with other tasks, maybe you need to add <eos token> at the end of the prompt:
  - use tokenizer.eos_token, tokenizer.eos_token_id to see eos_token of your model and correspond id
  - for ex: eos_token is "<|im_end|>". You need edit prompt like:
    - '''...The correct answer is {output_}.''' --> '''...The correct answer is {output_}. <|im_end|>'''
  - for ex: eos_token is "end_token__". You need edit prompt like:
    - '''...The correct answer is {output_}.''' --> '''...The correct answer is {output_}. end_token__'''

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    modelpath,
    padding_side="right",
    # add_eos_token=True,
    # add_bos_token=True,
    trust_remote_code=True,
    token = 'YOUR TOKEN HERE'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = "test thử mô hình"
tokens = tokenizer(prompt, return_tensors="pt")
print(tokens)
tokenizer.batch_decode(tokenizer.encode(prompt))

{'input_ids': tensor([[  1944, 131885, 130179, 128338]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


['test', ' thử', ' mô', ' hình']

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input'])):
        input_ = example['input'][i]
        output_ = example['output'][i]
        output_ = 'A. Positive' if output_ == 'positive' else 'B. Neutral' if output_ == 'neutral' else 'C. Negative'
        #text = f"### Question: {input__}\n ### Answer: {example['output'][i]}"
        text = f'''{input_}
A. Positive
B. Neutral
C. Negative

The correct answer is {output_}.'''

        output_texts.append(text)
    return output_texts

In [ ]:
dataset["validation"]['input'][0], dataset["validation"]['output'][0]

('The sentiment of this comment "Chất lượng vật chất kém." is', 'negative')

In [ ]:
formatting_prompts_func({'input': [dataset["validation"]['input'][0]],
                         'output': [dataset["validation"]['output'][0]]})

['The sentiment of this comment "Chất lượng vật chất kém." is\nA. Positive\nB. Neutral\nC. Negative\n\nThe correct answer is C. Negative.']

In [ ]:
print(formatting_prompts_func({'input': [dataset["validation"]['input'][0]],
                         'output': [dataset["validation"]['output'][0]]})[0])

The sentiment of this comment "Chất lượng vật chất kém." is
A. Positive
B. Neutral
C. Negative

The correct answer is C. Negative.


# Create model

## Load model
We use AutoModelForCausalLM.from_pretrained to load model:
- device_map = 'auto': auto active gpu.
- torch_dtype: use bfloat16, if your gpu don't support bfloat16, set it to float32
- attn_implementation: you can use 'flash_attention_2', if you meet bug, maybe your gpu doesn't support it, you need delete this line.
- token: you huggingface token like tokenizer above.

In [ ]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

model = AutoModelForCausalLM.from_pretrained(
    modelpath,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    #torch_dtype=torch.float32,
    #quantization_config=bnb_config,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
    token = 'YOUR TOKEN HERE'
)

In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2FlashAttention2(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Line

## Create peft
For lora training, we need create LoraConfig:
  - lora_alpha, r, dropout is basic hyperarameter.
  - almost LLM use bias = 'none' when finetune then we set is to None
  - target_modules are list of layer we want to finetune, I set it to 'all-linear', then all linear layers will be finetuned. If you just finetune some layers like q_proj, k_proj, you can pass list ['q_proj', 'k_proj'].
  - modules_to_save is other layers we want to finetune (but don't use lora), in my experience, finetune all embedding layers will make model work betters than I set modules_to_save as list of all embedding layers.

In [ ]:
#model.print_trainable_parameters()

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = 'all-linear',
#     target_modules=["q_proj",
#         "k_proj",
#         "v_proj",
#         "o_proj",
#         "gate_proj",
#         "up_proj",
#         "down_proj",
#         "lm_head",],
    modules_to_save=["embed_tokens", "rotary_emb"]
                     #"input_layernorm", "post_attention_layernorm", "norm"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#lora_model.print_trainable_parameters()

In [ ]:
#lora_model.print_trainable_parameters() #"embed_tokens"

In [ ]:
model.print_trainable_parameters() #

trainable params: 242,606,080 || all params: 1,786,320,384 || trainable%: 13.5813


## Eval model before training
We use create_prompt function to generate prompt (without output), our model will predict output. We will eval model before finetune. You can see some predict below.

You can see that pretrained model has poor performance

In [ ]:
def create_prompt(input_, output_):
    output_ = 'A. Positive' if output_ == 'positive' else 'B. Neutral' if output_ == 'neutral' else 'C. Negative'
        #text = f"### Question: {input__}\n ### Answer: {example['output'][i]}"
    text = f'''{input_}
A. Positive
B. Neutral
C. Negative

The correct answer is'''

    return text

sentence = 'món ăn rất ngon'
input_ = f'''The sentiment of this comment: "{sentence}" is'''
prompt = create_prompt(input_, "")
print(prompt)

inputs = torch.tensor([tokenizer.encode(prompt)])

tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=50,
    temperature=0.1,
    do_sample=False
)
print(tokenizer.decode(tokens[0], skip_special_tokens=False))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos toke

The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is
The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive.

The phrase "món ăn rất ngon" translates to "delicious food" in English, which indicates a positive sentiment towards the food being described. The use of "không" (not) at the end of the


In [ ]:
from datetime import datetime

start = datetime.now()

prediction = []
response = []
accuracy = []
labels = []

for i, x in df_test.iterrows():
    sentence = x['sentence']
    label = x['sentiment']
    input_ = f'''The sentiment of this comment: "{sentence}" is'''
    prompt = create_prompt(input_, label)

    inputs = tokenizer.encode(
        prompt,
        # add_generation_prompt=True,
        return_tensors='pt'
    )

    tokens = model.generate(
        inputs.to(model.device),
        max_new_tokens=3,
        temperature=0.1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    #break

    answer = tokenizer.decode(tokens[0], skip_special_tokens=False).split("The correct answer is ")[-1]
    answer = 'positive' if 'positive' in answer.lower() else 'negative' if 'negative' in answer.lower() else 'neutral'
    prediction.append(answer.lower())
    response.append(tokenizer.decode(tokens[0], skip_special_tokens=False))

    accuracy.append(prediction[-1] == label)
    labels.append(label)

    if i % 100 == 0:
        print(i, np.array(accuracy).sum()/len(prediction)*100, datetime.now() - start, (datetime.now() - start)/len(prediction))

0 100.0 0:00:00.121622 0:00:00.121624
100 69.3069306930693 0:00:12.179419 0:00:00.120588
200 70.64676616915423 0:00:24.347871 0:00:00.121134
300 67.77408637873754 0:00:36.492047 0:00:00.121236
400 68.5785536159601 0:00:48.678283 0:00:00.121392
500 69.26147704590818 0:01:00.780316 0:00:00.121318
600 67.55407653910149 0:01:12.851168 0:00:00.121217
700 66.76176890156918 0:01:24.998305 0:00:00.121253
800 67.41573033707866 0:01:37.215684 0:00:00.121368
900 66.48168701442842 0:01:49.254118 0:00:00.121259
1000 66.53346653346654 0:02:01.415417 0:00:00.121294
1100 66.03088101725703 0:02:13.598444 0:00:00.121343
1200 65.69525395503747 0:02:25.695514 0:00:00.121312
1300 66.10299769408148 0:02:37.724233 0:00:00.121233
1400 66.38115631691649 0:02:49.795400 0:00:00.121196
1500 66.35576282478348 0:03:01.936345 0:00:00.121210
1600 66.70830730793254 0:03:14.046403 0:00:00.121203
1700 66.84303350970018 0:03:26.051836 0:00:00.121136
1800 67.01832315380344 0:03:38.173664 0:00:00.121140
1900 66.96475539189

In [ ]:
from sklearn.metrics import classification_report, f1_score
import sklearn

print(sklearn.metrics.classification_report(labels, prediction, digits=4))

              precision    recall  f1-score   support

    negative     0.9503    0.9752    0.9626       686
     neutral     0.7647    0.0194    0.0378       670
    positive     0.5163    0.9985    0.6807       680

    accuracy                         0.6685      2036
   macro avg     0.7438    0.6644    0.5604      2036
weighted avg     0.7443    0.6685    0.5641      2036



In [ ]:
for x in response[-10:]:
    print(x)
    print("\n")

The sentiment of this comment: "The facilities of the university are versatile and helpful." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive


The sentiment of this comment: "Mấy bạn đó hay đòi hỏi nhưng không bao giờ giúp đỡ người khác." is
A. Positive
B. Neutral
C. Negative

The correct answer is C. Negative


The sentiment of this comment: "Cậu ấy rất có kỹ năng về sáng tạo và nghệ thuật." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive


The sentiment of this comment: "Giảng viên này không nhàm chán." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive


The sentiment of this comment: "Anh ta là một người rất tỉ mỉ và cẩn thận." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive


The sentiment of this comment: "Giáo viên đưa ra các phương tiện hỗ trợ giảng dạy rất tốt và hiệu quả." is
A. Positive
B. Neutral
C. Negative

The correct answer is A. Positive


The sentiment of this comment: "Th

## Create TrainingArguments

In [ ]:
print(len(df_train)//bs//ga_steps*epochs//4)

509


In [ ]:
save_step = len(df_train)//bs//ga_steps*epochs//4
print(save_step)

509


In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs_eval,
    evaluation_strategy="steps",
    eval_steps=save_step,
    gradient_accumulation_steps=ga_steps,
    optim="paged_adamw_32bit",
    save_steps=save_step,
    save_strategy="steps",
    logging_steps=save_step,
    learning_rate=lr,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    save_total_limit=1,
    #load_best_model_at_end=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Create trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    max_seq_length= 128,
    #dataset_text_field=["input", "output"],
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
    formatting_func = formatting_prompts_func,
    #data_collator=collator
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFT

Map:   0%|          | 0/8144 [00:00<?, ? examples/s]

Map:   0%|          | 0/2036 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


# Train

In [ ]:
trainer.train()

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Step,Training Loss,Validation Loss
509,0.783000,0.742862
1018,0.672800,0.740708
1527,0.600800,0.759789
2036,0.530600,0.793977


TrainOutput(global_step=2036, training_loss=0.6467708295361232, metrics={'train_runtime': 389.3988, 'train_samples_per_second': 83.657, 'train_steps_per_second': 5.229, 'total_flos': 1.2091384561287168e+16, 'train_loss': 0.6467708295361232, 'epoch': 4.0})

# Eval
- If you save the model to the output_dir folder, the training code above will automatically save the model to output_dir/checkpoint-{save_step} (in my case: out/checkpoint-2036). We will load the model from this folder:
  - I calculate _id = save_step * num_epoch = 509 * 4 = 2036
  - My model will saved at "{output_dir}/checkpoint-{_id}"
- I use del model, gc.collect() and torch.cuda.empty_cache() to release trained model (save gpu memory).
- We use PeftConfig.from_pretrained to lead peft config (this is the same as peft config at training)
- we load pretrained model like before.
- We load the tokenizer in this folder by passing the saved folder to AutoTokenizer.from_pretrained()
- We use PeftModel.from_pretrained(model, peft_model_id) to merge pre-trained model with finetuned lora model (note that when we use peft lora, training code only saves lora model, this saves our storage and saving time)

In [ ]:
save_step = len(df_train)//bs//ga_steps*epochs//4
print(save_step)

509


In [ ]:
import gc

del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

_id = save_step*4

peft_model_id = f"{output_dir}/checkpoint-{_id}"

config = PeftConfig.from_pretrained(peft_model_id)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=False,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

model = AutoModelForCausalLM.from_pretrained(
    modelpath,
    device_map="auto",
    #torch_dtype=torch.float16,
    torch_dtype=torch.bfloat16,
    #quantization_config=bnb_config,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
    token = 'YOUR TOKEN HERE'
)

tokenizer = AutoTokenizer.from_pretrained(f"{output_dir}/checkpoint-{_id}",
                                          trust_remote_code=True,
                                          padding_side='left',
                                          token='YOUR TOKEN HERE')

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def create_prompt(input_, output_):
    output_ = 'A. Positive' if output_ == 'positive' else 'B. Neutral' if output_ == 'neutral' else 'C. Negative'
        #text = f"### Question: {input__}\n ### Answer: {example['output'][i]}"
    text = f'''{input_}
A. Positive
B. Neutral
C. Negative

The correct answer is'''

    return text

In [ ]:
sentence = 'món ăn rất ngon'
input_ = f'''The sentiment of this comment: "{sentence}" is'''
prompt = create_prompt(input_, "")
print(prompt)

The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is


In [ ]:
inputs = torch.tensor([tokenizer.encode(prompt)])
inputs

tensor([[   785,  25975,    315,    419,   3980,     25,    330,     76,   3165,
         128442, 128323,   7777,    263,      1,    374,    198,     32,     13,
          43903,    198,     33,     13,  58694,    198,     34,     13,  50857,
            271,    785,   4396,   4226,    374]])

In [ ]:
tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=50,
    temperature=0.1,
    do_sample=True
)

In [ ]:
print(tokenizer.decode(tokens[0], skip_special_tokens=False))

The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is B. Neutral. The comment "môn ăn rất ngon" is a positive comment on the food served in the cafeteria. However, it does not provide any information about the quality of the food itself. Therefore, it cannot be classified as either


In [ ]:
tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=3,
    temperature=0.1,
    do_sample=False
)
print(tokenizer.decode(tokens[0], skip_special_tokens=False))

The sentiment of this comment: "món ăn rất ngon" is
A. Positive
B. Neutral
C. Negative

The correct answer is B. Neutral


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [ ]:
from datetime import datetime

start = datetime.now()

prediction = []
response = []
accuracy = []
labels = []

for i, x in df_test.iterrows():
    sentence = x['sentence']
    label = x['sentiment']
    input_ = f'''The sentiment of this comment: "{sentence}" is'''
    prompt = create_prompt(input_, label)

    inputs = tokenizer.encode(
        prompt,
        # add_generation_prompt=True,
        return_tensors='pt'
    )

    tokens = model.generate(
        inputs.to(model.device),
        max_new_tokens=3,
        temperature=0.1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    #break

    answer = tokenizer.decode(tokens[0], skip_special_tokens=False).split("The correct answer is ")[-1]
    answer = 'positive' if 'positive' in answer.lower() else 'negative' if 'negative' in answer.lower() else 'neutral'
    prediction.append(answer.lower())
    response.append(tokenizer.decode(tokens[0], skip_special_tokens=False))

    accuracy.append(prediction[-1] == label)
    labels.append(label)

    if i % 100 == 0:
        print(i, np.array(accuracy).sum()/len(prediction)*100, datetime.now() - start, (datetime.now() - start)/len(prediction))

0 100.0 0:00:00.117085 0:00:00.117088
100 87.12871287128714 0:00:11.459762 0:00:00.113463
200 86.06965174129353 0:00:22.732505 0:00:00.113097
300 87.37541528239203 0:00:34.022894 0:00:00.113033
400 85.53615960099751 0:00:45.343768 0:00:00.113077
500 85.82834331337325 0:00:56.750492 0:00:00.113274
600 86.02329450915141 0:01:08.133723 0:00:00.113367
700 85.73466476462197 0:01:19.426894 0:00:00.113305
800 86.14232209737828 0:01:30.704837 0:00:00.113240
900 86.12652608213097 0:01:42.046285 0:00:00.113259
1000 86.21378621378622 0:01:53.524557 0:00:00.113411
1100 86.28519527702089 0:02:04.847799 0:00:00.113395
1200 85.34554537885096 0:02:16.278358 0:00:00.113471
1300 85.54957724827055 0:02:27.701936 0:00:00.113530
1400 85.58172733761599 0:02:39.057269 0:00:00.113531
1500 85.27648234510326 0:02:50.385303 0:00:00.113515
1600 85.32167395377888 0:03:01.627965 0:00:00.113447
1700 85.71428571428571 0:03:12.845191 0:00:00.113372
1800 85.8411993337035 0:03:24.085562 0:00:00.113318
1900 85.6391372961

In [ ]:
from sklearn.metrics import classification_report, f1_score
import sklearn

print(sklearn.metrics.classification_report(labels, prediction, digits=4))

              precision    recall  f1-score   support

    negative     0.9714    0.9913    0.9812       686
     neutral     0.7972    0.7567    0.7764       670
    positive     0.7914    0.8147    0.8029       680

    accuracy                         0.8551      2036
   macro avg     0.8533    0.8542    0.8535      2036
weighted avg     0.8540    0.8551    0.8543      2036



# Check results
- I print all sentences have wrong prediction and see that almost cases is bug.
- Conclusion, dataset is not clean than finetune LLM can't better than finetune roberta (~89..90%), because roberta will overfit even in test dataset.
- In additionally, when I print example: "món ăn này rất ngon", we can see that model before finetune work better. Model after finetune only think about school (because finetune dataset is about school) and it don't know about food review. Than I think we only need finetune for special cases and finetune dataset need be clean and large enough.

In [ ]:
df_test['predict'] = prediction

In [ ]:
for i, row in df_test.iterrows():
    if row['predict'] != row['sentiment']:
        print(row['sentence'], row['predict'], row['sentiment'])

Giảng viên luôn đồng hành với sinh viên trên quá trình học tập. positive neutral
Tôi có thể ứng dụng các kiến thức có được từ chương trình đào tạo vào công việc của mình. neutral positive
Giảng viên rất tài năng và có nhiều kinh nghiệm trong công tác giảng dạy. positive neutral
Được học tập với các giáo viên giàu kinh nghiệm và thực tiễn. positive neutral
Nhà hàng và các cửa hàng tiện lợi ở gần trường rất đa dạng và phong phú. neutral positive
Chị ấy rất giỏi quản lý thời gian và luôn hoàn thành công việc đúng tiến độ. positive neutral
Anh ấy có một tầm nhìn đỉnh cao và khả năng giải quyết vấn đề tốt. positive neutral
Các cửa hàng tiện lợi gần đó thường xuyên được mở cửa cho sinh viên. negative neutral
Trường đại học này đóng góp một phần quan trọng trong việc phát triển kinh tế và xã hội địa phương. positive neutral
Có nhiều hoạt động ngoại khóa và phong phú cho sinh viên. neutral positive
Khu vực đặt máy bán thức uống rất tiện lợi cho sinh viên. neutral positive
Chương trình học giúp